# Loading Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk

In [31]:
"""I use percentile forcasting to project out the 10th percentile to 90th percentile outcome for baseball players entering the 
2021 season.""".replace("\n", "")

'I use percentile forcasting to project out the 10th percentile to 90th percentile outcome for baseball players entering the 2021 season.'

# Loading Data

In [2]:
forcast_data = pd.read_csv(r'C:\Users\ranih\OneDrive\Desktop\Python For Real\Github\Data\Baseball\Hitting\Fangraphs\2016-2019_custom_report_hitting_metrics_3.csv')
forcast_data.head()

,Name,Season,Age,G,PA,AB,R,HR,RBI,SB,...,EV,LA,Barrels,Barrel%,maxEV,HardHit%,Spd,wRC+,WAR,playerid
0,Mookie Betts,2018,25,136,614,520,129,32,80,30,...,92.3,18.5,57,13.10%,110.6,50.20%,7.0,185,10.4,13611
1,Mike Trout,2018,26,140,608,471,101,39,79,24,...,91.2,18.6,54,15.30%,118.0,46.00%,5.9,188,9.6,10155
2,Christian Yelich,2019,27,130,580,489,100,44,97,30,...,93.3,11.3,59,15.80%,117.9,48.70%,6.2,175,7.8,11477
3,Mike Trout,2017,25,114,507,402,92,33,72,22,...,88.8,18.0,38,12.00%,114.7,40.50%,6.2,180,6.8,10155
4,Mike Trout,2019,27,134,600,470,110,45,104,11,...,90.9,22.2,62,17.50%,116.6,43.80%,4.9,178,8.5,10155


In [3]:
print("Dataframe Rows: ", forcast_data.shape[0])
print("Dataframe Columns: ", forcast_data.shape[1])

Dataframe Rows:  1106
Dataframe Columns:  43


In [4]:
# grabbing the features we need
forcast_data =  forcast_data[['Name','Season','G','PA','AB','R','HR','RBI','SB','AVG']]
forcast_data.head()

,Name,Season,G,PA,AB,R,HR,RBI,SB,AVG
0,Mookie Betts,2018,136,614,520,129,32,80,30,0.346
1,Mike Trout,2018,140,608,471,101,39,79,24,0.312
2,Christian Yelich,2019,130,580,489,100,44,97,30,0.329
3,Mike Trout,2017,114,507,402,92,33,72,22,0.306
4,Mike Trout,2019,134,600,470,110,45,104,11,0.291


# Preparing the dataframe for forcasting

In [5]:
forcast_data = forcast_data[(forcast_data['Season'] >= 2017) & (forcast_data['Season'] < 2020)]
forcast_data.head()

,Name,Season,G,PA,AB,R,HR,RBI,SB,AVG
0,Mookie Betts,2018,136,614,520,129,32,80,30,0.346
1,Mike Trout,2018,140,608,471,101,39,79,24,0.312
2,Christian Yelich,2019,130,580,489,100,44,97,30,0.329
3,Mike Trout,2017,114,507,402,92,33,72,22,0.306
4,Mike Trout,2019,134,600,470,110,45,104,11,0.291


In [6]:
name_df_final = forcast_data['Name'].value_counts().to_frame().reset_index()
name_df_final.columns.values[0] = 'Name'
name_df_final.columns.values[1] = "seasons_of_data"
forcast_data = name_df_final.merge(forcast_data, how = 'inner', on = ['Name'])
forcast_data.head()

,Name,seasons_of_data,Season,G,PA,AB,R,HR,RBI,SB,AVG
0,Edwin Encarnacion,3,2017,157,669,554,96,38,107,2,0.258
1,Edwin Encarnacion,3,2019,109,486,418,81,34,86,0,0.244
2,Edwin Encarnacion,3,2018,137,579,500,74,32,107,3,0.246
3,Chad Pinder,3,2018,110,333,298,43,13,27,0,0.258
4,Chad Pinder,3,2017,87,309,282,36,15,42,2,0.238


In [7]:
forcast_data_grouped = forcast_data.groupby(['Name','Season']).sum()
forcast_data_grouped = forcast_data[forcast_data['seasons_of_data'] >= 3]
forcast_data_grouped = forcast_data_grouped.groupby(['Name','Season']).sum()
forcast_data_grouped.head()

seasons_of_data    G   PA   AB    R  HR  RBI  SB    AVG
Name         Season                                                         
A.J. Pollock 2017                  3  112  466  425   73  14   49  20  0.266
             2018                  3  113  460  413   61  21   65  13  0.257
             2019                  3   86  342  308   49  15   47   5  0.266
Aaron Judge  2017                  3  155  678  542  128  52  114   9  0.284
             2018                  3  112  498  413   77  27   67   6  0.278

In [8]:
fantasy_cats_restind = forcast_data_grouped.reset_index()
fantasy_cats_restind.head(3)

,Name,Season,seasons_of_data,G,PA,AB,R,HR,RBI,SB,AVG
0,A.J. Pollock,2017,3,112,466,425,73,14,49,20,0.266
1,A.J. Pollock,2018,3,113,460,413,61,21,65,13,0.257
2,A.J. Pollock,2019,3,86,342,308,49,15,47,5,0.266


# Percentile Forcasting

In [9]:
fantasy_cats_restind.drop(['Season','seasons_of_data'], axis=1 ,inplace=True)

In [10]:
fantasy_cats_restind.head(3)

,Name,G,PA,AB,R,HR,RBI,SB,AVG
0,A.J. Pollock,112,466,425,73,14,49,20,0.266
1,A.J. Pollock,113,460,413,61,21,65,13,0.257
2,A.J. Pollock,86,342,308,49,15,47,5,0.266


In [11]:
len(fantasy_cats_restind)/3

152.0

In [12]:
# data cuts
counter = 0
player_index = 0
left = 0
right = 3
col = 0

# lists
at_bats_lst = []
r_run_lst = []
b_home_run_lst = []
b_rbi_lst = []
stolen_bases_lst = []
b_avg_lst = []

for index in range(len(fantasy_cats_restind)):
    
    player_name = fantasy_cats_restind.iloc[left:right, 0]
    at_bat = fantasy_cats_restind.iloc[left:right, 3]
    r_run = fantasy_cats_restind.iloc[left:right, 4]
    b_home_run = fantasy_cats_restind.iloc[left:right, 5]
    r_rbi = fantasy_cats_restind.iloc[left:right, 6]
    stolen_base =  fantasy_cats_restind.iloc[left:right, 7]
    batting_avg = fantasy_cats_restind.iloc[left:right, 8]
    
    ab_percentiles = np.percentile(at_bat, [10, 20, 30, 40, 50, 60, 70, 80, 90])
    ab_percentiles = np.around(ab_percentiles)
    
    run_percentiles = np.percentile(r_run, [10, 20, 30, 40, 50, 60, 70, 80, 90])
    run_percentiles = np.around(run_percentiles)
    
    hr_percentiles = np.percentile(b_home_run, [10, 20, 30, 40, 50, 60, 70, 80, 90])
    hr_percentiles = np.around(hr_percentiles)
    
    rbi_percentiles = np.percentile(r_rbi, [10, 20, 30, 40, 50, 60, 70, 80, 90])
    rbi_percentiles = np.around(rbi_percentiles)
    
    sb_percentiles = np.percentile(stolen_base, [10, 20, 30, 40, 50, 60, 70, 80, 90])
    sb_percentiles = np.around(sb_percentiles)
    
    ba_percentiles = np.percentile(batting_avg, [10, 20, 30, 40, 50, 60, 70, 80, 90])
    ba_percentiles = np.around(ba_percentiles,3)
    
    print(fantasy_cats_restind.columns[0], player_name[player_index])
    
    at_bats_perc = list(ab_percentiles)
    run_perc = list(run_percentiles)
    homerun_perc = list(hr_percentiles)
    rbi_perc = list(rbi_percentiles)
    stolen_bases_perc = list(sb_percentiles)
    batting_avg_perc = list(ba_percentiles)
    
    print(fantasy_cats_restind.columns[3], at_bats_perc)
    at_bats_lst.append(at_bats_perc)
    print(fantasy_cats_restind.columns[4], run_perc)
    r_run_lst.append(run_perc)
    print(fantasy_cats_restind.columns[5],  homerun_perc)
    b_home_run_lst.append(homerun_perc)
    print(fantasy_cats_restind.columns[6], rbi_perc)
    b_rbi_lst.append(rbi_perc)
    print(fantasy_cats_restind.columns[7], stolen_bases_perc)
    stolen_bases_lst.append(stolen_bases_perc)
    print(fantasy_cats_restind.columns[8], batting_avg_perc)
    b_avg_lst.append(batting_avg_perc)
    
    
    print()
    print()

    
    left += 3
    right += 3
    player_index += 3
    counter += 1
    if counter == len(fantasy_cats_restind)/3:
        print("end")
        break

Name A.J. Pollock
AB [329.0, 350.0, 371.0, 392.0, 413.0, 415.0, 418.0, 420.0, 423.0]
R [51.0, 54.0, 56.0, 59.0, 61.0, 63.0, 66.0, 68.0, 71.0]
HR [14.0, 14.0, 15.0, 15.0, 15.0, 16.0, 17.0, 19.0, 20.0]
RBI [47.0, 48.0, 48.0, 49.0, 49.0, 52.0, 55.0, 59.0, 62.0]
SB [7.0, 8.0, 10.0, 11.0, 13.0, 14.0, 16.0, 17.0, 19.0]
AVG [0.259, 0.261, 0.262, 0.264, 0.266, 0.266, 0.266, 0.266, 0.266]


Name Aaron Judge
AB [385.0, 392.0, 399.0, 406.0, 413.0, 439.0, 465.0, 490.0, 516.0]
R [75.0, 76.0, 76.0, 77.0, 77.0, 87.0, 97.0, 108.0, 118.0]
HR [27.0, 27.0, 27.0, 27.0, 27.0, 32.0, 37.0, 42.0, 47.0]
RBI [57.0, 60.0, 62.0, 65.0, 67.0, 76.0, 86.0, 95.0, 105.0]
SB [4.0, 4.0, 5.0, 5.0, 6.0, 7.0, 7.0, 8.0, 8.0]
AVG [0.273, 0.274, 0.276, 0.277, 0.278, 0.279, 0.28, 0.282, 0.283]


Name Adam Frazier
AB [336.0, 353.0, 371.0, 388.0, 406.0, 436.0, 465.0, 495.0, 524.0]
R [53.0, 53.0, 54.0, 54.0, 55.0, 60.0, 65.0, 70.0, 75.0]
HR [7.0, 8.0, 8.0, 9.0, 10.0, 10.0, 10.0, 10.0, 10.0]
RBI [38.0, 41.0, 44.0, 47.0, 50.0, 51.0,

Name Miguel Rojas
AB [314.0, 356.0, 399.0, 441.0, 483.0, 484.0, 485.0, 486.0, 487.0]
R [38.0, 40.0, 41.0, 43.0, 44.0, 46.0, 47.0, 49.0, 50.0]
HR [2.0, 3.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0, 10.0]
RBI [30.0, 34.0, 38.0, 42.0, 46.0, 47.0, 49.0, 50.0, 52.0]
SB [3.0, 4.0, 4.0, 5.0, 6.0, 7.0, 7.0, 8.0, 8.0]
AVG [0.258, 0.265, 0.271, 0.278, 0.284, 0.285, 0.286, 0.288, 0.289]


Name Mike Moustakas
AB [529.0, 536.0, 542.0, 549.0, 555.0, 559.0, 562.0, 566.0, 569.0]
R [68.0, 70.0, 71.0, 73.0, 75.0, 76.0, 77.0, 78.0, 79.0]
HR [29.0, 31.0, 32.0, 34.0, 35.0, 36.0, 36.0, 37.0, 37.0]
RBI [85.0, 86.0, 86.0, 87.0, 87.0, 89.0, 90.0, 92.0, 93.0]
SB [1.0, 1.0, 2.0, 2.0, 3.0, 3.0, 3.0, 4.0, 4.0]
AVG [0.252, 0.252, 0.253, 0.253, 0.254, 0.258, 0.261, 0.265, 0.268]


Name Mike Trout
AB [416.0, 429.0, 443.0, 456.0, 470.0, 470.0, 470.0, 471.0, 471.0]
R [94.0, 96.0, 97.0, 99.0, 101.0, 103.0, 105.0, 106.0, 108.0]
HR [34.0, 35.0, 37.0, 38.0, 39.0, 40.0, 41.0, 43.0, 44.0]
RBI [73.0, 75.0, 76.0, 78.0, 79.0, 84.0, 89.0, 

In [13]:
flat_list_at_bats = [item for sublist in at_bats_lst for item in sublist]
flat_list_runs = [item for sublist in r_run_lst for item in sublist]
flat_list_home_runs = [item for sublist in b_home_run_lst for item in sublist]
flat_list_rbis = [item for sublist in b_rbi_lst for item in sublist]
flat_list_sb = [item for sublist in stolen_bases_lst for item in sublist]
flat_list_avg = [item for sublist in b_avg_lst for item in sublist]

In [26]:
player_names = fantasy_cats_restind['Name'].unique()
player_names_lst = player_names.tolist()
player_names_lst = player_names_lst * 9
player_names_lst.sort()

In [15]:
percentiles_list = ['10th percentile', '20th percentile', '30th percentile', '40th percentile', '50th percentile', '60th percentile', '70th percentile','80th percentile','90th percentile']
percentiles_list = percentiles_list * 2645

In [21]:
player_names_df = pd.DataFrame(player_names_lst, columns=["Name"])
percentiles_list_df = pd.DataFrame(percentiles_list, columns=["Percentile"])
at_bat_lst_df = pd.DataFrame(flat_list_at_bats, columns= ['ABs'])
r_run_lst_df = pd.DataFrame(flat_list_runs, columns= ['Runs'])
b_home_run_lst_df = pd.DataFrame(flat_list_home_runs, columns= ['HRs'])
b_rbi_lst_df = pd.DataFrame(flat_list_rbis, columns= ['RBIs'])
b_stolen_bases_lst_df = pd.DataFrame(flat_list_sb, columns= ['SBs'])
b_avg_lst_df = pd.DataFrame(flat_list_avg, columns=['BA'])

In [22]:
percentiles_df = pd.concat([player_names_df, percentiles_list_df, at_bat_lst_df, r_run_lst_df, b_home_run_lst_df, b_rbi_lst_df, b_stolen_bases_lst_df, b_avg_lst_df], axis=1,
                         )
percentiles_df = percentiles_df.dropna()

In [27]:
percentiles_df_grp = percentiles_df.groupby(['Name','Percentile']).sum()

In [36]:
final_df = percentiles_df_grp.copy().reset_index()

In [32]:
player_name = input("Enter a players name to see thier projections: ")
final_df[final_df['Name'] == player_name]

Enter a players name to see thier projections: Kris Bryant


,Name,Percentile,ABs,Runs,HRs,RBIs,SBs,BA
810,Kris Bryant,10th percentile,420.0,69.0,16.0,56.0,2.0,0.274
811,Kris Bryant,20th percentile,451.0,79.0,19.0,60.0,3.0,0.276
812,Kris Bryant,30th percentile,481.0,88.0,23.0,65.0,3.0,0.278
813,Kris Bryant,40th percentile,512.0,98.0,26.0,69.0,4.0,0.280
814,Kris Bryant,50th percentile,543.0,108.0,29.0,73.0,4.0,0.282
815,Kris Bryant,60th percentile,544.0,109.0,29.0,74.0,5.0,0.285
816,Kris Bryant,70th percentile,545.0,109.0,30.0,75.0,5.0,0.287
817,Kris Bryant,80th percentile,547.0,110.0,30.0,75.0,6.0,0.290
818,Kris Bryant,90th percentile,548.0,110.0,31.0,76.0,6.0,0.292
